# GPS位置上游门架检索

In [12]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

PATH_FOLDER_CPRN_PAYLOAD : str = r'/Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/'
MODELNAME_CPRN : str = 'CPRN_DG_GTR_EMBD_JS_HDMAP_V413_250911_b7c61dfffb57a88a1e2c301ea0a7dbd682e82351c333dd98af037b26ad46fae4.pkl'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load CPRN Model

In [13]:
from cprn.data.pickle import PickleIO
dg_cprn = PickleIO.load_from_pickle(PATH_FOLDER_CPRN_PAYLOAD + MODELNAME_CPRN)
print(dg_cprn)

File hash checking : b7c61dfffb57a88a1e2c301ea0a7dbd682e82351c333dd98af037b26ad46fae4 : passed 
DiGraph with 219803 nodes and 223647 edges


## Retrieve Facility List

In [16]:
from cprn.model.topo.topo_search import CprnTopoSearch as CTS
df_facs = CTS.list_fac_df(dg_cprn)

print(len(df_facs))
df_facs.sample(2)

12041


,vtx_fac,fac_code,fac_type,fac_name,fac_ghz,rrl_id_hdm,rrl_id_osm
8438,WTTBT53PT3YU001450,WTTBT53PT3YU001450,DC2,1|2,NaN,NaN,NaN
1009,WTTDUP9M94TX001297,WTTDUP9M94TX001297,DC1,3|1,NaN,NaN,NaN


In [21]:
df_facs.query(" fac_code == 'S003032001000710010' ")

,vtx_fac,fac_code,fac_type,fac_name,fac_ghz,rrl_id_hdm,rrl_id_osm
11994,WTTMB65GDQGX001289,S003032001000710010,G1,滨江-常州高新区枢纽1,WTTMB65GBC2N-00000,NaN,1333767355


In [22]:
dg_cprn.nodes['WTTMB65GDQGX001289']['lst_fac_attr']

[{'vtx_fac': 'WTTMB65GDQGX001289',
  'fac_code': 'S003032001000710010',
  'fac_type': 'G1',
  'fac_name': '滨江-常州高新区枢纽1',
  'fac_ghz': 'WTTMB65GBC2N-00000',
  'rrl_id_hdm': nan,
  'rrl_id_osm': '1333767355'}]

In [23]:
# facs = {'G1'}

dg_cprn.nodes['WTTMB65GDQGX001289'].get('lst_fac_attr')

[{'vtx_fac': 'WTTMB65GDQGX001289',
  'fac_code': 'S003032001000710010',
  'fac_type': 'G1',
  'fac_name': '滨江-常州高新区枢纽1',
  'fac_ghz': 'WTTMB65GBC2N-00000',
  'rrl_id_hdm': nan,
  'rrl_id_osm': '1333767355'}]

## 设施检索

In [25]:
list(dg_cprn.successors('WTTMB65GDQGX001289'))

['WTTM8PY7C2NV001289']

In [63]:
# lst_fac_search = CTS.fac_bfs_depth_v2(DG = dg_cprn, start_node = 'WTTMB65GDQGX001289', 
lst_fac_search = CTS.fac_bfs_depth_v2(DG = dg_cprn, start_node = 'WTTM8PY7C2NV001289', 
    fac_types = ['G1','G2','G3'],  direction = 'downstream', 
    max_depth = 3, max_dist = 15000, mark_max_dist = True,
    query_avoid_fac=['S003932002000210010'],
    query_avoid_edge = "rtype in ['SA'] or (knd == 'ST' and cls == 'MR')",
    verbose = True)

len(lst_fac_search)

import pandas as pd
df =  pd.DataFrame(lst_fac_search)
# df.query("reach_max_dist != True")

2025-09-15 17:54:00.861 | INFO     | cprn.model.topo.topo_search:fac_bfs_depth_v2:72 - 🏰 Facility S003932001000410010 at vtx WTTJXCN5MP80001648 found
2025-09-15 17:54:00.862 | INFO     | cprn.model.topo.topo_search:fac_bfs_depth_v2:72 - 🏰 Facility S003932001000320010 at vtx WTTJZGFTNP89001676 found
2025-09-15 17:54:00.863 | INFO     | cprn.model.topo.topo_search:fac_bfs_depth_v2:72 - 🏰 Facility S003932001000220020 at vtx WTTJZZ17PEKH001606 found
2025-09-15 17:54:00.864 | INFO     | cprn.model.topo.topo_search:fac_bfs_depth_v2:95 - 🚫 Avoid Edge WTTJZVECZWUP001441 -> WTTJZVEFPFG3001450, {'rrl_id': 87162080.0, 'weight': 2.44, 'rcode': 'S39', 'knd': 'ST', 'cls': 'MR', 'rtype': 'NA', 'mdir': 'NA', 'lane': 2.0}
2025-09-15 17:54:00.867 | INFO     | cprn.model.topo.topo_search:fac_bfs_depth_v2:95 - 🚫 Avoid Edge WTTNPW01MB0H001223 -> WTTNPQR8PFFF001142, {'rrl_id': 44586590.0, 'weight': 141.81, 'rcode': '', 'knd': 'FW', 'cls': 'SA', 'rtype': 'SA', 'mdir': 'BWD', 'lane': 2.0}
2025-09-15 17:54:00.

In [62]:
df

,depth,vtx_intvl_src,vtx_intvl_tgt,interval_weight,cumulative_weight,reach_max_depth,vtx_fac,fac_code,fac_type,fac_name,fac_ghz,rrl_id_hdm,rrl_id_osm,reach_max_dist
0,1,WTTM8PY7C2NV001289,WTTJXCN5MP80001648,5169.29,5169.29,False,WTTJXCN5MP80001648,S003932001000410010,G1,春江至罗溪枢纽1,WTTJXCN5D4F6-00000,34138946.0,NaN,NaN
1,1,WTTM8PY7C2NV001289,WTTJZGFTNP89001676,4635.75,4635.75,False,WTTJZGFTNP89001676,S003932001000320010,G1,春江至孟河1,WTTJZGFV2UZS-00000,99674547.0,NaN,NaN
2,2,WTTJZGFTNP89001676,WTTJZZ17PEKH001606,1950.10,6585.85,False,WTTJZZ17PEKH001606,S003932001000220020,G1,孟河至丹阳新桥2,WTTJZZ1E4E4E-00000,43449844.0,NaN,NaN
3,2,WTTJXCN5MP80001648,WTTM29FJ1428001282,6679.64,11848.93,False,WTTM29FJ1428001282,G004232001000920010,G1,罗溪枢纽至薛家1,WTTM29CURBDN-00000,47325835.0,NaN,NaN
4,2,WTTJZZ17PEKH001606,WTTNR5C5KGZG001108,8513.55,15099.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
5,2,WTTJXCN5MP80001648,WTTJPWJUK7UT001457,6803.53,11972.82,False,WTTJPWJUK7UT001457,S003932002000310010,G1,龙城大道至邹区1,WTTJPWJU99DX-00000,47912279.0,NaN,NaN
6,2,WTTJXCN5MP80001648,WTTJW9HKXD8Y001573,7837.44,13006.73,False,WTTJW9HKXD8Y001573,G004232001001010010,G1,罗溪枢纽至罗墅湾1,WTTJW9HT2SBC-00000,38470495.0,NaN,NaN
7,2,WTTJW9HKXD8Y001573,WTTJW4R20NB1001175,2016.36,15023.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
8,2,WTTJW9HKXD8Y001573,WTTJW4Q7B1YE001720,1995.34,15002.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
9,2,WTTM29FJ1428001282,WTTM1R31J6TF001400,3416.22,15265.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [53]:
df

,depth,vtx_intvl_src,vtx_intvl_tgt,interval_weight,cumulative_weight,vtx_fac,fac_code,fac_type,fac_name,fac_ghz,rrl_id_hdm,rrl_id_osm,reach_max_depth,reach_max_dist
0,0,WTTMB65GDQGX001289,WTTMB65GDQGX001289,0.00,0.00,WTTMB65GDQGX001289,S003032001000710010,G1,滨江-常州高新区枢纽1,WTTMB65GBC2N-00000,NaN,1333767355,NaN,NaN
1,1,WTTMB65GDQGX001289,WTTJXCN5MP80001648,6748.90,6748.90,WTTJXCN5MP80001648,S003932001000410010,G1,春江至罗溪枢纽1,WTTJXCN5D4F6-00000,34138946.0,NaN,False,NaN
2,1,WTTMB65GDQGX001289,WTTJZGFTNP89001676,6215.36,6215.36,WTTJZGFTNP89001676,S003932001000320010,G1,春江至孟河1,WTTJZGFV2UZS-00000,99674547.0,NaN,False,NaN
3,2,WTTJZGFTNP89001676,WTTJZZ17PEKH001606,1950.10,8165.46,WTTJZZ17PEKH001606,S003932001000220020,G1,孟河至丹阳新桥2,WTTJZZ1E4E4E-00000,43449844.0,NaN,False,NaN
4,2,WTTJXCN5MP80001648,WTTM29FJ1428001282,6679.64,13428.54,WTTM29FJ1428001282,G004232001000920010,G1,罗溪枢纽至薛家1,WTTM29CURBDN-00000,47325835.0,NaN,False,NaN
5,2,WTTJZZ17PEKH001606,WTTNR1XR4E79001657,6986.74,15152.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
6,2,WTTJXCN5MP80001648,WTTJPWJUK7UT001457,6803.53,13552.43,WTTJPWJUK7UT001457,S003932002000310010,G1,龙城大道至邹区1,WTTJPWJU99DX-00000,47912279.0,NaN,False,NaN
7,2,WTTJXCN5MP80001648,WTTJW9HKXD8Y001573,7837.44,14586.34,WTTJW9HKXD8Y001573,G004232001001010010,G1,罗溪枢纽至罗墅湾1,WTTJW9HT2SBC-00000,38470495.0,NaN,False,NaN
8,2,WTTJW9HKXD8Y001573,WTTJW93KH7X0001470,426.84,15013.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
9,2,WTTM29FJ1428001282,WTTM2BV6F3V5001351,1606.01,15034.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


## BACKWARD SEARCH

In [64]:
vtx_gps_location = 'WTSX6BBZP2J6000857'
vtx_gps_location = 'WTSX7H38GM88001434'
# vtx_gps_location = 'WTSMR7JG972F002103'
vtx_gps_location = 'WTSMX5UCTHMW001202'

lst_fac_search = CTS.fac_bfs_depth_v2(DG = dg_cprn, start_node = vtx_gps_location, 
    fac_types = ['G1','G2','G3'],  direction = 'upstream', 
    max_depth = 1, max_dist = 10000, mark_max_dist = True,
    # query_avoid_edge = "rtype in ['SA'] or (knd == 'ST' and cls == 'MR')",
    verbose = True)

import pandas as pd
df_res = pd.DataFrame(lst_fac_search)
# df.query("is_fac  == True")
df_res

2025-09-15 18:00:18.058 | INFO     | cprn.model.topo.topo_search:fac_bfs_depth_v2:72 - 🏰 Facility G250332003000820010 at vtx WTSMR4Z6XWRK001801 found
2025-09-15 18:00:18.059 | INFO     | cprn.model.topo.topo_search:fac_bfs_depth_v2:53 - 📛 Stop Criteria Activated: Traverse distance exceeds 10000, stop searching at WTSMPSN0Q4UF001336
2025-09-15 18:00:18.059 | INFO     | cprn.model.topo.topo_search:fac_bfs_depth_v2:72 - 🏰 Facility G250332003000710010 at vtx WTSMRDGTW7GT001815 found
2025-09-15 18:00:18.060 | INFO     | cprn.model.topo.topo_search:fac_bfs_depth_v2:53 - 📛 Stop Criteria Activated: Traverse distance exceeds 10000, stop searching at WTSMQEX3KTGG003042
2025-09-15 18:00:18.060 | INFO     | cprn.model.topo.topo_search:fac_bfs_depth_v2:53 - 📛 Stop Criteria Activated: Traverse distance exceeds 10000, stop searching at WTSMRG6DE3PQ001344


,depth,vtx_intvl_src,vtx_intvl_tgt,interval_weight,cumulative_weight,reach_max_depth,vtx_fac,fac_code,fac_type,fac_name,fac_ghz,rrl_id_hdm,rrl_id_osm,reach_max_dist
0,1,WTSMX5UCTHMW001202,WTSMR4Z6XWRK001801,7903.98,7903.98,True,WTSMR4Z6XWRK001801,G250332003000820010,G1,东善桥至南庄1,WTSMR4Z6P6PR-00000,49228445.0,NaN,NaN
1,0,WTSMX5UCTHMW001202,WTSMPSN0Q4UF001336,10079.35,10079.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,1,WTSMX5UCTHMW001202,WTSMRDGTW7GT001815,9094.85,9094.85,True,WTSMRDGTW7GT001815,G250332003000710010,G1,殷巷至南庄1,WTSMRDGWJZZS-00000,100163504.0,NaN,NaN
3,1,WTSMR4Z6XWRK001801,WTSMQEX3KTGG003042,2139.79,10043.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,1,WTSMRDGTW7GT001815,WTSMRG6DE3PQ001344,932.71,10027.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [24]:
lst_fac_search[0]['fac_types']

{'G1'}

In [65]:
df_fac_search = CTS.parse_fac_interval_df_v2(lst_fac_search, vtx_gps_location, dg_cprn)

df_fac_search.T

KeyError: "['gh_fac_rp', 'geom_fac_wgs_3d'] not in index"

In [ ]:
## 下游顶点检索示例

# 使用新添加的 get_downstream_vertices 方法
# 选择一个GPS位置作为起点
vtx_start = 'WTSMX5UCTHMW001202'

print(f"起点节点: {vtx_start}")
print(f"节点是否存在: {vtx_start in dg_cprn}")

# 获取直接下游顶点
downstream_direct = CTS.get_downstream_vertices(dg_cprn, vtx_start, max_depth=1)
print(f"\n直接下游顶点 ({len(downstream_direct)}个):")
for i, vtx in enumerate(downstream_direct[:10]):  # 只显示前10个
    print(f"  {i+1}. {vtx}")

# 获取2层深度的下游顶点
downstream_depth2 = CTS.get_downstream_vertices(dg_cprn, vtx_start, max_depth=2)
print(f"\n2层深度下游顶点 ({len(downstream_depth2)}个):")
for i, vtx in enumerate(downstream_depth2[:10]):  # 只显示前10个
    print(f"  {i+1}. {vtx}")

# 获取下游顶点（包含权重信息）
downstream_with_weights = CTS.get_downstream_vertices(dg_cprn, vtx_start, max_depth=1, include_weights=True)
print(f"\n直接下游顶点（含权重）({len(downstream_with_weights)}个):")
for i, (vtx, weight) in enumerate(downstream_with_weights[:10]):  # 只显示前10个
    print(f"  {i+1}. {vtx} (权重: {weight})")


In [ ]:
## 上游顶点检索示例

# 使用新添加的 get_upstream_vertices 方法
print(f"起点节点: {vtx_start}")

# 获取直接上游顶点
upstream_direct = CTS.get_upstream_vertices(dg_cprn, vtx_start, max_depth=1)
print(f"\n直接上游顶点 ({len(upstream_direct)}个):")
for i, vtx in enumerate(upstream_direct[:10]):  # 只显示前10个
    print(f"  {i+1}. {vtx}")

# 获取2层深度的上游顶点
upstream_depth2 = CTS.get_upstream_vertices(dg_cprn, vtx_start, max_depth=2)
print(f"\n2层深度上游顶点 ({len(upstream_depth2)}个):")
for i, vtx in enumerate(upstream_depth2[:10]):  # 只显示前10个
    print(f"  {i+1}. {vtx}")

# 获取上游顶点（包含权重信息）
upstream_with_weights = CTS.get_upstream_vertices(dg_cprn, vtx_start, max_depth=1, include_weights=True)
print(f"\n直接上游顶点（含权重）({len(upstream_with_weights)}个):")
for i, (vtx, weight) in enumerate(upstream_with_weights[:10]):  # 只显示前10个
    print(f"  {i+1}. {vtx} (权重: {weight})")


In [ ]:
## 性能对比：新方法 vs 原有方法

import time

# 测试节点
test_node = 'WTSMX5UCTHMW001202'

print("=== 性能对比测试 ===")
print(f"测试节点: {test_node}")

# 测试新方法 - 直接下游顶点
start_time = time.time()
downstream_new = CTS.get_downstream_vertices(dg_cprn, test_node, max_depth=1)
time_new = time.time() - start_time
print(f"\n新方法 (get_downstream_vertices, max_depth=1):")
print(f"  耗时: {time_new:.4f}秒")
print(f"  结果数量: {len(downstream_new)}")

# 测试新方法 - 2层深度下游顶点
start_time = time.time()
downstream_new_depth2 = CTS.get_downstream_vertices(dg_cprn, test_node, max_depth=2)
time_new_depth2 = time.time() - start_time
print(f"\n新方法 (get_downstream_vertices, max_depth=2):")
print(f"  耗时: {time_new_depth2:.4f}秒")
print(f"  结果数量: {len(downstream_new_depth2)}")

# 测试原有方法 - 设施BFS搜索（只搜索1层深度）
start_time = time.time()
fac_search_old = CTS.fac_bfs_depth(DG=dg_cprn, start_node=test_node, 
                                  fac_types=['G1','G2','G3'], direction='downstream', 
                                  max_depth=1, max_dist=10000, verbose=False)
time_old = time.time() - start_time
print(f"\n原有方法 (fac_bfs_depth, max_depth=1):")
print(f"  耗时: {time_old:.4f}秒")
print(f"  结果数量: {len(fac_search_old)}")

print(f"\n=== 性能提升 ===")
print(f"1层深度: 新方法比原有方法快 {time_old/time_new:.2f} 倍")
print(f"2层深度: 新方法处理了 {len(downstream_new_depth2)} 个顶点")

# 显示部分结果对比
print(f"\n=== 结果对比 ===")
print(f"新方法直接下游顶点 (前5个): {downstream_new[:5]}")
if fac_search_old:
    print(f"原有方法找到的设施节点: {[item.get('vtx_intvl_tgt', 'N/A') for item in fac_search_old[:5]]}")
else:
    print("原有方法未找到符合条件的设施节点")
